In [ ]:
%python

import dlt
from pyspark import SparkSession
from pyspark.sql.functions import *

bigqueryproject = "batch-processing-de"
schema_name = "e_com"

tables_list = ["bronze_orders","bronze_customers","bronze_orders","bronze_products","silver_orders","silver_sales","silver_products"]

def load_to_bigquery(
        bigqueryproject: str,
        schema_name: str,
        table_name: str,
        location_path: str
):
        if "bronze" in table_name:
            partition_col = "bronze_ingestion_timestamp"
            print(f"Reading data from {table_name}--------------------")
            df = dlt.read(bigqueryproject+'.'+schema_name+'.'+table_name)
            print(f"Writing data into {location_path}------------------")
            return(
                df.write.mode("overwrite").partitionBy(to_date(partition_col)).parquet(location_path)
            )
        else:
            partition_col = "silver_processed_timestamp"
            df = dlt.read(bigqueryproject+'.'+schema_name+'.'+table_name)
            return(
                df.write.mode("overwrite").partitionBy(to_date(partition_col)).parquet(location_path)
            )

if __name__ == "__main__":
    for i in tables_list:
        location_path = f"gs://batch-processing-de_dlt_storage/dlt_stagging_data/{i}"
        load_to_bigquery(bigqueryproject, schema_name, i, location_path)
        print(f"Data written into {location_path} for table {i}-----------------")

